In [137]:
import pandas as pd
import numpy as np

import numba

import bebi103

import altair as alt
import altair_catplot as altcat

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

In [138]:
df = pd.read_csv('../data/hw_4.2_caulobacter_growth_image_processing_results.csv')

df.head()

,time (min),area (sq um),growth_event,bacterium
0,1.0,1.300624,0,1
1,2.0,1.314144,0,1
2,3.0,1.295216,0,1
3,4.0,1.314144,0,1
4,5.0,1.341184,0,1


Since the length of division is different for all growth events we should add another column that restarts the time count from 0 every time there's a division.

In [139]:
time = []
j = 0
for i in df['growth_event'].diff():
    if i == 0:
        j += 1
        time.append(j)
    else:
        j = 0
        time.append(j)

df['t'] = time

In [140]:
df.head()

,time (min),area (sq um),growth_event,bacterium,t
0,1.0,1.300624,0,1,0
1,2.0,1.314144,0,1,1
2,3.0,1.295216,0,1,2
3,4.0,1.314144,0,1,3
4,5.0,1.341184,0,1,4


Let's start with a subset of the data. We will look at bacterium 1.

In [141]:
df_bacterium1 = df.loc[df['bacterium'] == 1]

In [142]:
p = bokeh.plotting.figure(plot_width=650,
                          plot_height=250,
                          x_axis_label='time (min)',
                          y_axis_label='cell area (sq µm)')

# Specify the glyphs
colors = ['#1f78b4', '#a6cee3']
for i, g in df_bacterium1.groupby('growth_event'):
    p.circle(g['time (min)'], g['area (sq um)'], size=3, color=colors[i%2])

bokeh.io.show(p)

#### No hierarchy

Let's just model a single growth event for now with no hierarchy using the exponential model.

In [7]:
def data_prior_pred(t):
    '''
    Samples parameter values according to the prior and generates
    data y at the values given in t.
    '''
    # Sample parameter values according to priors
    a = np.random.normal(1.2, 0.4)
    k = np.random.normal(0.01, 0.003)
    sigma = np.abs(np.random.normal(0, 0.1))
    
    # Generate random data according to the likelihood
    return np.random.normal(a * np.exp(k * t), sigma)

In [8]:
p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'new time'].values

# Plot simulated data
for i in range(100):
    p.circle(t, data_prior_pred(t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

In [9]:
p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 3, 'new time'].values

# Plot simulated data
for i in range(100):
    p.circle(t, data_prior_pred(t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 3, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

Now let's try the linear model.

In [10]:
def data_prior_pred_linear(t):
    '''
    Samples parameter values according to the prior and generates
    data y at the values given in t.
    '''
    # Sample parameter values according to priors
    a = np.random.normal(1.2, 0.4)
    b = np.random.normal(0.01, 0.003)
    sigma = np.abs(np.random.normal(0, 0.1))
    
    # Generate random data according to the likelihood
    return np.random.normal(a + b * t, sigma)

In [11]:
p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'new time'].values

# Plot simulated data
for i in range(100):
    p.circle(t, data_prior_pred_linear(t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

#### One level hierarchical model

Let's try a one level hierarchical model for bacterium 1.

In [18]:
model_code_pri_pred = """
data {
  // Number of data points for each experiment
  int N;
  // Number of entries in each level of the hierarchy
  int J_1;
  // Input of parameters of the priors
  real hyper_a0_mu;
  real hyper_a0_sigma;
  
  real hyper_k0_mu;
  real hyper_k0_sigma;
  
  real hyper_a0_tau;
  real hyper_k0_tau;
}


generated quantities {
  // Total number of data points 
  real a[N * J_1];
  real k[N * J_1];
  
  // Priors
  real a0 = normal_rng(hyper_a0_mu, hyper_a0_sigma);
  real k0 = normal_rng(hyper_k0_mu, hyper_k0_sigma);
  
  real a0_sigma = fabs(normal_rng(0, hyper_a0_sigma));
  real k0_sigma = fabs(normal_rng(0, hyper_k0_sigma));
  
  real a0_tau = fabs(normal_rng(0, hyper_a0_tau));
  real k0_tau = fabs(normal_rng(0, hyper_k0_tau));
  
  // Second layer
  real a_1[J_1]; 
  real k_1[J_1];
  
  for (i in 1:J_1) {
    a_1[i] = normal_rng(a0, a0_tau);
    k_1[i] = normal_rng(k0, k0_tau);
    for (j in 1:N) {
      a[(i - 1) * N + j] = normal_rng(a_1[i], a0_sigma);
      k[(i - 1) * N + j] = normal_rng(k_1[i], k0_sigma);
    }
  }
    
  }
"""

In [19]:
# Compile
sm_gen = bebi103.stan.StanModel(model_code=model_code_pri_pred)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_fe32cb4498718f51dde97b497b28c319 NOW.
/Users/madelynw/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/cn/x4zmqr494ln7sfp5z2kytzqw0000gp/T/tmps7xwt2t3/stanfit4anon_model_fe32cb4498718f51dde97b497b28c319_4258100889973328742.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [77]:
# Number of iteration
N_iter = 1000

# 98 data points per experiments, 3 growth events
data = dict(N=98,
            J_1=3,
            hyper_a0_mu=1.4,
            hyper_a0_sigma=0.3,
            hyper_k0_mu=0.01,
            hyper_k0_sigma=0.002,
            hyper_a0_tau=0.1,
            hyper_k0_tau=0.001)

# Sample
df_pred = sm_gen.sampling(data=data,
                     algorithm='Fixed_param',
                     warmup=0,
                     chains=1,
                     iter=N_iter)

df_samples_a = bebi103.stan.extract_array(df_pred, name='a')
df_samples_k = bebi103.stan.extract_array(df_pred, name='k')

p_a = bebi103.viz.predictive_ecdf(df_pred, 
                                'a', 
                                x_axis_label='a')
p_k = bebi103.viz.predictive_ecdf(df_pred, 
                                'k', 
                                x_axis_label='k')
# Plot
bokeh.io.show(bokeh.layouts.gridplot([p_a, p_k], ncols=2))

Check for negative values:

In [78]:
len(df_samples_a[df_samples_a['a'] < 0]) / len(df_samples_a)

0.003343537414965986

In [79]:
len(df_samples_k[df_samples_k['k'] < 0]) / len(df_samples_k)

0.0020816326530612244

In [80]:
len(df_samples_a)

294000

Prior predictive checks

In [81]:
a = df_samples_a['a'].values
k = df_samples_k['k'].values

p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'new time'].values

# Plot simulated data
for i in range(0, len(df_samples_a), 1000):
    p.circle(t, a[i] * np.exp(k[i] * t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

Trying the same priors/samples for the linear model.

In [82]:
a = df_samples_a['a'].values
k = df_samples_k['k'].values

p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'new time'].values

# Plot simulated data
for i in range(0, len(df_samples_a), 1000):
    p.circle(t, a[i] + k[i] * t, size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

Noncentered model code:

In [143]:
model_code_linear_noncentered = """
data {
  // Total number of data points
  int N;
  
  // Number of entries in each level of the hierarchy
  int J_1;

  //Index arrays to keep track of hierarchical structure
  int index_1[N];
  
  // The measurements
  real area[N];
  
  // Time
  vector[N] t;
}


parameters {
  // Hyperparameters level 0
  real a;
  real k;
  real<lower=0> sigma;

  // How hyperparameters vary
  real<lower=0> tau_a;
  real<lower=0> tau_k;

  // Hyperparameters level 1
  vector[J_1] a_1_tilde;
  vector[J_1] k_1_tilde;
}

transformed parameters {
  // Transformations for noncentered
  vector[J_1] a_1 = a + tau_a * a_1_tilde;
  vector[J_1] k_1 = k + tau_k * k_1_tilde;
}

model {
  a ~ normal(1.4, 0.3);
  k ~ normal(0.01, 0.002);
  sigma ~ normal(0, 0.1);
  tau_a ~ normal(0, 0.1);
  tau_k ~ normal(0, 0.001);

  a_1_tilde ~ normal(0, 0.1);
  k_1_tilde ~ normal(0, 0.001);

  area ~ normal(a_1[index_1] + k_1[index_1] .* t, sigma);
}
"""

In [144]:
sm_noncentered = bebi103.stan.StanModel(model_code=model_code_linear_noncentered)

Using cached StanModel.


In [145]:
# Rename for convenience
df_bacterium1 = df_bacterium1.rename(columns={'area (sq um)': 'area'})

df_bacterium1.head()

,time (min),area,growth_event,bacterium,t
0,1.0,1.300624,0,1,0
1,2.0,1.314144,0,1,1
2,3.0,1.295216,0,1,2
3,4.0,1.314144,0,1,3
4,5.0,1.341184,0,1,4


In [146]:
data, df_part = bebi103.stan.df_to_datadict_hier(df_bacterium1,
                                           level_cols=['growth_event'],
                                           data_cols=['area', 't'])

# Take a look
data

{'N': 1888,
 'J_1': 20,
 'index_1': array([ 1,  1,  1, ..., 20, 20, 20]),
 'area': array([1.300624, 2.019888, 2.00096 , ..., 1.644032, 1.600768, 2.355184]),
 't': array([ 0, 71, 70, ..., 21, 19, 80])}

In [148]:
# Sample
samples_linear = sm_noncentered.sampling(data=data, seed=2389412)

# Convert to data frame for easy use later
df_linear = bebi103.stan.to_dataframe(samples_linear)

bebi103.stan.check_all_diagnostics(samples_linear)

n_eff / iter looks reasonable for all parameters.
Rhat looks reasonable for all parameters.
0 of 4000 (0.0%) iterations ended with a divergence.
58 of 4000 (1.45%) iterations saturated the maximum tree depth of 10.
  Try running again with max_treedepth set to a larger value to avoid saturation.
E-BFMI indicated no pathological behavior.


8

In [149]:
bokeh.io.show(bebi103.viz.corner(samples_linear, pars=['a', 'k']))

In [150]:
# Marginalized distributions of each parameter 
plots = [bebi103.viz.ecdf(df_linear[param], x_axis_label=param, plot_height=200, plot_width=250) 
                 for param in ['a', 'k']]
bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))

How to do posterior predictive checks?

In [156]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples_linear, name='a', data=df_bacterium1['area']))

RuntimeError: column 'a' is either absent or scalar-valued.

Noncentered exponential model

In [164]:
model_code_exp_noncentered = """
data {
  // Total number of data points
  int N;
  
  // Number of entries in each level of the hierarchy
  int J_1;

  //Index arrays to keep track of hierarchical structure
  int index_1[N];
  
  // The measurements
  real area[N];
  
  // Time
  vector[N] t;
}


parameters {
  // Hyperparameters level 0
  real a;
  real k;
  real<lower=0> sigma;

  // How hyperparameters vary
  real<lower=0> tau_a;
  real<lower=0> tau_k;

  // Hyperparameters level 1
  vector[J_1] a_1_tilde;
  vector[J_1] k_1_tilde;
}

transformed parameters {
  // Transformations for noncentered
  vector[J_1] a_1 = a + tau_a * a_1_tilde;
  vector[J_1] k_1 = k + tau_k * k_1_tilde;
}

model {
  a ~ normal(1.4, 0.3);
  k ~ normal(0.01, 0.002);
  sigma ~ normal(0, 0.1);
  tau_a ~ normal(0, 0.1);
  tau_k ~ normal(0, 0.001);

  a_1_tilde ~ normal(0, 0.1);
  k_1_tilde ~ normal(0, 0.001);

  area ~ normal(a_1[index_1] .* exp(k_1[index_1] .* t), sigma);
}
"""

In [165]:
sm_exp = bebi103.stan.StanModel(model_code=model_code_exp_noncentered)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_e98c33e3ebe18409fcc0d2831790a5aa NOW.
/Users/madelynw/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/cn/x4zmqr494ln7sfp5z2kytzqw0000gp/T/tmp2ug0l2qv/stanfit4anon_model_e98c33e3ebe18409fcc0d2831790a5aa_32417326029689983.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [166]:
# Sample
samples_exp = sm_noncentered.sampling(data=data, seed=2389412)

# Convert to data frame for easy use later
df_exp = bebi103.stan.to_dataframe(samples_exp)

bebi103.stan.check_all_diagnostics(samples_exp)

n_eff / iter looks reasonable for all parameters.
Rhat looks reasonable for all parameters.
0 of 4000 (0.0%) iterations ended with a divergence.
58 of 4000 (1.45%) iterations saturated the maximum tree depth of 10.
  Try running again with max_treedepth set to a larger value to avoid saturation.
E-BFMI indicated no pathological behavior.


8

In [167]:
bokeh.io.show(bebi103.viz.corner(samples_exp, pars=['a', 'k']))

In [175]:
bokeh.io.show(bebi103.viz.predictive_ecdf(samples_exp, name='area', data=df_bacterium1['area']))

RuntimeError: column 'area' is either absent or scalar-valued.

In [122]:
%load_ext watermark

In [123]:
%watermark -v -p numpy,scipy,bokeh,jupyterlab

CPython 3.7.0
IPython 7.1.1

numpy 1.15.4
scipy 1.1.0
bokeh 1.0.1
jupyterlab 0.35.3
